# ADF4351 Functional test   - with FTDI or FX2LP
- https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4351.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from utilities.adapters import peripherals
from signal_generators.adf435x import ADF4351
from signal_generators.adf435x.fx2 import AnalogDevicesFX2LP
from utilities.shift_register import ShiftRegister

Populating the interactive namespace from numpy and matplotlib
No USB device matches URL ftdi://ftdi:ft232h/1


In [5]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

## Debug mode?

In [6]:
cls = ADF4351 

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [7]:
with_hardware_device = False

if with_hardware_device:
    _clk = peripherals.Pin.get_Ftdi_pin(pin_id = 4)
    _data = peripherals.Pin.get_Ftdi_pin(pin_id = 1)
    _ss = peripherals.Pin.get_Ftdi_pin(pin_id = 3) 
    _spi = ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 0)
    
else:
    _spi = _ss = None  # using None for testing without actual hardware device.
    
bus = peripherals.SPI(_spi, _ss)


****** Virtual device. Data may not be real ! ******



In [8]:
bus = AnalogDevicesFX2LP()  # un-mark this to use FX2LP


****** Virtual device. Data may not be real ! ******



In [9]:
adf = cls(bus)  

In [10]:
ad_1500M = [0x3C0000, 0x80087D1, 0x30041C2, 0xE404B3, 0x932224, 0X580005] 
my_config_1500M = [0x3C0000, 134252497, 50348482, 14943411, 9642532, 5767173] 

df = adf.map.compare_values_sets_pd(enumerate(ad_1500M), enumerate(my_config_1500M))
df[df.different == 1]

,register,address,default_value,element_name,idx_lowest_bit,n_bits,read_only,value,value_2,different


In [11]:
# adf.map.load_values(enumerate(ad_1500M))
# adf.write_all_registers()

In [12]:
# adf.map.load_values(enumerate(my_config_1500M))
# adf.write_all_registers()

In [13]:
adf.set_frequency(2e9, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [14]:
adf.set_frequency(1.5e9, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [15]:
adf.set_frequency(1.0e9, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [16]:
adf.rf_n_divider.step(3)

In [17]:
adf.set_frequency(500e6, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [18]:
adf.set_frequency(435e6, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [19]:
adf.set_frequency(434375000, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [20]:
adf.set_frequency(50.2e6, channel_resolution=100e3)
# adf.current_dividers

In [21]:
ad_50M = [0x400000, 0x80087D1, 0x3004FC2, 0x6004B3, 0xEC8224, 0x580005]
my_config_50M = [4194304, 134252497, 50352066, 6292659, 15499812, 5767173]
my_config_50_2M = [4195328, 134252497, 50351682, 1203, 15499812, 5767173]

df = adf.map.compare_values_sets_pd(enumerate(ad_50M), enumerate(my_config_50_2M))
df[df.different == 1]

,register,address,default_value,element_name,idx_lowest_bit,n_bits,read_only,value,value_2,different
2,REGISTER_0,0,0,FRAC,3,12,False,0,128,1
18,REGISTER_2,2,0,LDF,8,1,False,1,0,1
19,REGISTER_2,2,0,LDP,7,1,False,1,0,1
27,REGISTER_3,3,0,ABP,22,1,False,1,0,1
28,REGISTER_3,3,0,Charge_Cancelation,21,1,False,1,0,1


In [22]:
adf.map.load_values(enumerate(ad_1500M))
adf.write_all_registers()

In [23]:
adf.map.load_values(enumerate(my_config_50_2M))
adf.write_all_registers()

## BFSK test

In [24]:
adf.reset()

adf.rf_n_divider._set_channel_resolution(100e3)

reg_0_1500M = 0x3C0000 + 0x40
reg_0_1500_2M = 0x3C0020
reg_values = (reg_0_1500M, reg_0_1500_2M)
seq = np.random.choice((0, 1), size = 100)

for s in seq:
    adf.map._registers[0].load_value(reg_values[s])
    adf._write_register_0()

## Current Configuration

In [25]:
RF_out_min = 2.2e9 / 64
RF_out_max = 4.4e9
RF_out_min, RF_out_max

(34375000.0, 4400000000.0)

In [26]:
# adf.set_frequency(RF_out_min, channel_resolution = 100e3, rf_divider_as = None)
adf.set_frequency(35e6, channel_resolution = 100e3, rf_divider_as = None)
# adf.set_frequency(1.599899e9) 
# adf.set_frequency(RF_out_max, channel_resolution = 100e3, rf_divider_as = None)

# adf.phaser.set_phase(90)

df_dividers, df_controls = adf.current_configuration
df_dividers

,type,source_type,source_freq,my_divider,divider_equivalent,is_integer,my_freq
0,_ReferenceInput,NaN,NaN,NaN,NaN,NaN,2.500000e+07
1,_ReferenceDoubler,_ReferenceInput,2.500000e+07,2.0,NaN,True,5.000000e+07
2,_R_Counter,_ReferenceDoubler,5.000000e+07,1.0,NaN,True,5.000000e+07
3,_ReferenceDivider,_R_Counter,5.000000e+07,2.0,NaN,True,2.500000e+07
4,_RF_N_Divider,_VCO,2.240000e+09,89.6,89.6,False,2.500000e+07
5,_VCO,_ReferenceDivider,2.500000e+07,89.6,NaN,False,2.240000e+09
6,_RF_Divider,_VCO,2.240000e+09,64.0,NaN,True,3.500000e+07
7,_RF_Output,_RF_Divider,3.500000e+07,1.0,NaN,True,3.500000e+07
8,_AuxOutput,_VCO,2.240000e+09,1.0,NaN,True,2.240000e+09


In [27]:
df_controls

,type,LDF,LDP,CSR,inverted_polarity,source_type,source_freq,enabled,mode,my_divider,is_integer,my_freq,phase_adjust_enabled,phase,charge_pump_current,three_state_enabled,cancelation_enabled
0,_PhaseFrequencyDetector,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,_BandSelectClockDivider,NaN,NaN,NaN,NaN,_ReferenceDivider,25000000.0,True,HIGH,50.0,True,500000.000000,NaN,NaN,NaN,NaN,NaN
2,_MuxOut,NaN,NaN,NaN,NaN,THREE_STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,_Phaser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.44,NaN,NaN,NaN
4,_ClockDivider,NaN,NaN,NaN,NaN,_ReferenceDivider,25000000.0,NaN,CLOCK_DIVIDER_OFF,150.0,True,166666.666667,NaN,NaN,NaN,NaN,NaN
5,_NoiseControl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW_NOISE_MODE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,_ChargePump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.31,False,False


## Find Integer N-dividers for a Frequency

In [28]:
adf.find_integer_N_dividers(freq_desired = 1.500e9,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

[((2, 1, 2, 120.0, 2),
  (25000000.0,
   50000000.0,
   50000000.0,
   25000000.0,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2, 2, 2, 240.0, 2),
  (25000000.0,
   50000000.0,
   25000000.0,
   12500000.0,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2, 3, 2, 360.0, 2),
  (25000000.0,
   50000000.0,
   16666666.666666666,
   8333333.333333333,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2, 4, 2, 480.0, 2),
  (25000000.0,
   50000000.0,
   12500000.0,
   6250000.0,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2, 5, 2, 600.0, 2),
  (25000000.0,
   50000000.0,
   10000000.0,
   5000000.0,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2, 6, 2, 720.0, 2),
  (25000000.0,
   50000000.0,
   8333333.333333333,
   4166666.6666666665,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2, 7, 2, 840.0, 2),
  (25000000.0,
   50000000.0,
   7142857.142857143,
   3571428.5714285714,
   3000000000.0,
   1500000000.0,
   1500000000.0)),
 ((2,

In [29]:
adf.find_integer_N_dividers(freq_desired = 50e6,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

[((2, 1, 2, 128.0, 64),
  (25000000.0,
   50000000.0,
   50000000.0,
   25000000.0,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 2, 2, 256.0, 64),
  (25000000.0,
   50000000.0,
   25000000.0,
   12500000.0,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 3, 2, 384.0, 64),
  (25000000.0,
   50000000.0,
   16666666.666666666,
   8333333.333333333,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 4, 2, 512.0, 64),
  (25000000.0,
   50000000.0,
   12500000.0,
   6250000.0,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 5, 2, 640.0, 64),
  (25000000.0,
   50000000.0,
   10000000.0,
   5000000.0,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 6, 2, 768.0, 64),
  (25000000.0,
   50000000.0,
   8333333.333333333,
   4166666.6666666665,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 7, 2, 896.0, 64),
  (25000000.0,
   50000000.0,
   7142857.142857143,
   3571428.5714285714,
   3200000000.0,
   50000000.0,
   50000000.0)),
 ((2, 8, 2, 1024.0, 64),
 

## Member functions test

In [30]:
# for f in dir(cls):
#     if not f.startswith('_'):
#         print('adf.{}()'.format(f))

In [31]:
adf.init()

In [32]:
adf.start()

In [33]:
adf.reset() 

# adf.set_frequency(1.5002e9, channel_resolution = 100e3)
adf.set_frequency(1.5002e9, channel_resolution = 100e3, rf_divider_as = 2)

df_dividers, df_controls = adf.current_configuration
print(adf.rf_n_divider.MOD)
df_dividers

250


,type,source_type,source_freq,my_divider,divider_equivalent,is_integer,my_freq
0,_ReferenceInput,NaN,NaN,NaN,NaN,NaN,2.500000e+07
1,_ReferenceDoubler,_ReferenceInput,2.500000e+07,2.000,NaN,True,5.000000e+07
2,_R_Counter,_ReferenceDoubler,5.000000e+07,1.000,NaN,True,5.000000e+07
3,_ReferenceDivider,_R_Counter,5.000000e+07,2.000,NaN,True,2.500000e+07
4,_RF_N_Divider,_VCO,3.000400e+09,120.016,120.016,False,2.500000e+07
5,_VCO,_ReferenceDivider,2.500000e+07,120.016,NaN,False,3.000400e+09
6,_RF_Divider,_VCO,3.000400e+09,2.000,NaN,True,1.500200e+09
7,_RF_Output,_RF_Divider,1.500200e+09,1.000,NaN,True,1.500200e+09
8,_AuxOutput,_VCO,3.000400e+09,1.000,NaN,True,3.000400e+09


In [34]:
adf.current_dividers

{'d_ref_doubler': 2,
 'd_r_counter': 1,
 'd_ref_divider': 2,
 'd_rf_n_divider': 120.016,
 'd_rf_divider': 2}

In [35]:
adf.current_frequency

1500200000.0

In [36]:
adf.current_phase

1.44

In [37]:
adf.enable(False)

In [38]:
adf.enable(True)

In [39]:
adf.enable_output(False)

In [40]:
adf.enable_output(True)

In [41]:
# adf.enable_output_channel(0)

In [42]:
adf.enabled

True

In [43]:
adf.freq_pfd

25000000.0

In [44]:
adf.freq_resolution

100000.0

In [45]:
adf.phase_resolution

1.44

In [46]:
adf.is_virtual_device

True

In [47]:
adf.load_registers(enumerate(ad_1500M))
adf.write_all_registers()

In [48]:
adf.power_downed

False

In [49]:
# adf.read_all_registers()

In [50]:
adf.registers_values

[(0, 3932160),
 (1, 134252497),
 (2, 50348482),
 (3, 14943411),
 (4, 9642532),
 (5, 5767173)]

In [51]:
# adf.select_freq_source(0)

In [52]:
# adf.select_phase_source(0)

In [53]:
adf.set_dividers(2, 1, 2, 120, 2)

In [54]:
adf.set_frequency(1.5002e9)

In [55]:
adf.set_phase(0)

In [56]:
adf.update()

In [57]:
# adf.status

In [58]:
adf.print()


<< REGISTER_0 >>  :  ('0x3c0020', '0b1111000000000000100000')
[ Reserved_31 ] :  0
[ INT ]         :  120
[ FRAC ]        :  4
[ Index ]       :  0

<< REGISTER_1 >>      :  ('0x80007d1', '0b1000000000000000011111010001')
[ Reserved_29 ]     :  0
[ Phase_Adjust ]    :  0
[ Prescaler_Value ] :  1
[ Phase_Value ]     :  0
[ MOD ]             :  250
[ Index ]           :  1

<< REGISTER_2 >>                   :  ('0x3004042', '0b11000000000100000001000010')
[ Reserved_31 ]                  :  0
[ Low_Noise_and_Low_Spur_Modes ] :  0
[ MUXOUT ]                       :  0
[ Reference_Doubler ]            :  1
[ RDIV2 ]                        :  1
[ R_Counter ]                    :  1
[ Double_Buffer ]                :  0
[ Charge_Pump_Current_Setting ]  :  0
[ LDF ]                          :  0
[ LDP ]                          :  0
[ Phase_Detector_Polarity ]      :  1
[ Power_Down ]                   :  0
[ Charge_Pump_Three_State ]      :  0
[ Counter_Reset ]                :  0
[ Index 

In [59]:
adf.pause()

In [60]:
adf.resume()

In [61]:
adf.stop()

In [62]:
adf.close()

## Testing sub-components

In [63]:
# for c in ['noise_control', 'charge_pump', 'muxout', 'mclk', 'ref_doubler', 'r_counter', 'ref_divider', 'phase_frequency_detector', 
#  'vco', 'rf_divider', 'rf_out', 'auxout', 'band_select_clock_divider', 'clock_divider','rf_n_divider', 'phaser']:
    
#     for f in dir(getattr(adf, c)):
#         if not f.startswith('__') :
# #             if f.islower():
#                 if callable(getattr(getattr(adf, c), f)):
#                     print('adf.{}.{}()'.format(c, f))

### noise_control

In [64]:
adf.noise_control._set_mode(mode = 'LOW_NOISE_MODE')

### charge_pump

In [65]:
adf.charge_pump._enable_cancelation(False)

In [66]:
adf.charge_pump._enable_cancelation(True)

In [67]:
adf.charge_pump._enable_three_state(False)

In [68]:
adf.charge_pump._set_current(0.31)

### mclk

In [69]:
adf.mclk.set_frequency(25e6)

### ref_doubler

In [70]:
adf.ref_doubler._by_2(True)

In [71]:
adf.ref_divider._by_2(True)

### phase_frequency_detector

In [72]:
adf.phase_frequency_detector._enable_cycle_slip_reduction(True)

In [73]:
adf.phase_frequency_detector._enable_lock_detect_function(True)

In [74]:
adf.phase_frequency_detector._enable_lock_detect_precision(True)

In [75]:
adf.phase_frequency_detector._set_antibacklash_pulse_width(True)

In [76]:
adf.phase_frequency_detector._set_lock_detect_pin_operation('DIGITAL_LOCK_DETECT')

In [77]:
adf.phase_frequency_detector._set_polarity(non_inverting = True)

### vco

In [78]:
adf.vco._power_down(False)

In [79]:
adf.vco.set_frequency(3.0e9)

True

### rf_divider

In [80]:
adf.rf_divider._enable_double_buffered(False)

In [81]:
adf.rf_divider._set_divider(2)

True

### rf_out

In [82]:
adf.rf_out._mute_till_lock_detected(True)

In [83]:
adf.rf_out.enable(True)

In [84]:
adf.rf_out.set_frequency(1.5e9)

True

In [85]:
adf.rf_out.set_output_power(5)

### auxout

In [86]:
adf.auxout._set_input_source('DIVIDED')

In [87]:
adf.auxout.enable(True) 

In [88]:
adf.auxout.set_output_power(2)

### band_select_clock_divider

In [89]:
adf.band_select_clock_divider._enable_band_select(True)

In [90]:
adf.band_select_clock_divider._refresh_divider()

('HIGH', 50, 50)

In [91]:
adf.band_select_clock_divider._set_mode('HIGH')

### clock_divider

In [92]:
adf.clock_divider._set_mode('CLOCK_DIVIDER_OFF')

### rf_n_divider

In [93]:
adf.rf_n_divider._set_channel_resolution(100e3)

In [94]:
adf.rf_n_divider._set_divider(120) 

True

In [95]:
adf.rf_n_divider._set_prescaler('8/9')

In [96]:
adf.rf_n_divider._synch_freq_pfd()

In [97]:
adf.rf_n_divider.set_frequency(adf.ref_divider.freq)

True

In [98]:
adf.rf_n_divider.step(1)

### phaser

In [99]:
adf.phaser._enable_phase_adjust(False)

In [100]:
adf.phaser.set_phase(90)